In [1]:
from pathlib import Path
import numpy as np
import os
import pandas as pd
import re
import time

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/UniCon/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [3]:
import os
import pandas as pd

# Define the directory path for the attack folders
input_dir = '../data/can-ml/2011-chevrolet-impala/post-attack-labeled'  # Replace with your directory path
output_dir = '../data/can-ml/2011-chevrolet-impala/merged'  # Replace with the output directory path for merged files

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List of attack types (folder names)
attack_types = ['combined-attacks', 'DoS-attacks', 'fuzzing-attacks', 'gear-attacks',
                'interval-attacks', 'rpm-attacks', 'speed-attacks', 'standstill-attacks', 'systematic-attacks']

# Loop through each attack type
for attack_type in attack_types:
    attack_folder = os.path.join(input_dir, attack_type)
    
    # Check if the folder exists
    if not os.path.isdir(attack_folder):
        print(f"Folder {attack_folder} does not exist, skipping.")
        continue
    
    # Initialize a list to store DataFrames
    df_list = []
    
    # Loop through each CSV file in the folder
    for file_name in os.listdir(attack_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(attack_folder, file_name)
            
            # Read the CSV file
            df = pd.read_csv(file_path)
            
            # Data preprocessing
            # Convert arbitration_id to integer (decimal) if it's a valid value
            df['arbitration_id'] = df['arbitration_id'].apply(lambda x: int(x, 16) if isinstance(x, str) else x)
            
            # Convert data_field to binary format with fixed length of 64 bits if it's a valid string
            df['data_field'] = df['data_field'].apply(lambda x: bin(int(x, 16))[2:].zfill(64) if isinstance(x, str) else x)
            
            # Remove rows with any invalid values
            df = df[['timestamp', 'arbitration_id', 'data_field', 'attack']].dropna()

            # Add the DataFrame to the list
            df_list.append(df)
    
    # Merge all DataFrames in the list
    merged_df = pd.concat(df_list, ignore_index=True)
    
    # Save the merged DataFrame to a new CSV file
    attack_type_cleaned = attack_type.replace("-attacks", "")
    output_file = os.path.join(output_dir, f'{attack_type_cleaned}.csv')
    merged_df.to_csv(output_file, index=False)
    
    print(f"Merged {attack_type_cleaned} saved to {output_file}")


Merged combined saved to ../data/can-ml/2011-chevrolet-impala/merged/combined.csv
Merged DoS saved to ../data/can-ml/2011-chevrolet-impala/merged/DoS.csv
Merged fuzzing saved to ../data/can-ml/2011-chevrolet-impala/merged/fuzzing.csv
Merged gear saved to ../data/can-ml/2011-chevrolet-impala/merged/gear.csv
Merged interval saved to ../data/can-ml/2011-chevrolet-impala/merged/interval.csv
Merged rpm saved to ../data/can-ml/2011-chevrolet-impala/merged/rpm.csv
Merged speed saved to ../data/can-ml/2011-chevrolet-impala/merged/speed.csv
Merged standstill saved to ../data/can-ml/2011-chevrolet-impala/merged/standstill.csv
Merged systematic saved to ../data/can-ml/2011-chevrolet-impala/merged/systematic.csv


In [4]:
# Define the directory path
directory_path = '../data/can-ml/2011-chevrolet-impala/merged/fuzzing.csv'

df = pd.read_csv(directory_path)
# Display the first few rows of the combined DataFrame
filtered_df = df[df['attack'] != 0]
filtered_df

,timestamp,arbitration_id,data_field,attack
49323,1.672531e+09,36,0000000000000000011010101000001111010010010111...,1
49340,1.672531e+09,172,1111001000001000001110010110101111000101101110...,1
49352,1.672531e+09,2013,0000000000000000000000000000000000000000000000...,1
49364,1.672531e+09,1119,0000000000000000000000000000000000100010010100...,1
49381,1.672531e+09,693,0000000000000000111110111001000111010110011101...,1
...,...,...,...,...
1248799,1.672531e+09,922,1100011001011111001011100110110100000110110100...,1
1248800,1.672531e+09,560,0000000000000000000000000111110101010101111100...,1
1248802,1.672531e+09,457,1110101011001100011000010100001110011000110101...,1
1248806,1.672531e+09,531,0000000000000000100110111110110110000111011111...,1


In [1]:
import os
import pandas as pd

# Directory path containing CSV files
# /home/hieutt/UniCon/data/can-ml/2017-subaru-forester/merged
input_dir = '../data/can-ml/2017-subaru-forester/merged'  # Change the path if needed

# List of CSV files in the directory
files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# Function to check each file
def check_data(file_path):
    print(f"\nChecking file: {file_path}")
    
    # Read the file
    df = pd.read_csv(file_path)
    
    # Display the shape of the DataFrame
    print("Shape:", df.shape)
    
    # Display the first 5 rows
    print("Head:")
    display(df.head())
    
    # Check for NaN values
    nan_count = df.isna().sum().sum()
    if nan_count > 0:
        print("Warning: There are NaN values in the data.")
        print(df.isna().sum())
    else:
        print("No NaN values found.")
    
    # Check data types
    print("\nData types:")
    print(df.dtypes)
    
    # Check label distribution if 'label' column exists
    if 'label' in df.columns:
        print("\nLabel distribution:")
        print(df['label'].value_counts())
    
    # Descriptive statistics for numeric columns
    print("\nDescriptive statistics for numeric columns:")
    print(df.describe())
    
# Iterate over each file and check
for file in files:
    check_data(os.path.join(input_dir, file))



Checking file: ../data/can-ml/2017-subaru-forester/merged/combined.csv
Shape: (5806825, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,356,0000001000000000000000000000000000000000000000...,0
1,1.672531e+09,881,0000000000000000000000001100000011111110000111...,0
2,1.672531e+09,328,0000101100010100010100001000001001000001000000...,0
3,1.672531e+09,329,0111111000000001001111111001100110101010001100...,0
4,1.672531e+09,117,0000000000000000000000000000000000000000000010...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  5.806825e+06    5.806825e+06  5.806825e+06
mean   1.672531e+09    4.111147e+02  1.039036e-02
std    1.636970e+02    3.333655e+02  1.014022e-01
min    1.672531e+09    2.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/DoS.csv
Shape: (2297897, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,342,0000000110000000000000000000000000000000000000...,0
1,1.672531e+09,320,0000000000000100101010000000100111010010110100...,0
2,1.672531e+09,321,0011101000110000100001110010111110101000110010...,0
3,1.672531e+09,864,0100000000010100100010011000001100110001011100...,0
4,1.672531e+09,865,0000000000111010000000001101010000001000000000...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  2.297897e+06    2.297897e+06  2.297897e+06
mean   1.672532e+09    4.054529e+02  2.254148e-02
std    2.930624e+02    3.377873e+02  1.484364e-01
min    1.672531e+09    0.000000e+00  0.000000e+00
25%    1.672531e+09    2.100000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/speed.csv
Shape: (3672497, 4)
Head:


...

No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  2.623432e+06    2.623432e+06  2.623432e+06
mean   1.672531e+09    4.051509e+02  4.052478e-02
std    1.451882e+02    3.309302e+02  1.971865e-01
min    1.672531e+09    2.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/systematic.csv
Shape: (3397184, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,320,0000000000000001001001000000100000101000001010...,0
1,1.672531e+09,321,1110001100101010001111110010101100100100100010...,0
2,1.672531e+09,324,0000000000000011011101110011100000110000101010...,0
3,1.672531e+09,328,0000101100010000001000011000100001000001000000...,0
4,1.672531e+09,329,0001011000001100001111110010100110101010000111...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  3.397184e+06    3.397184e+06  3.397184e+06
mean   1.672531e+09    4.165107e+02  6.579567e-03
std    1.962385e+02    3.403088e+02  8.084725e-02
min    1.672531e+09    0.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    2.047000e+03  1.000000e+00

Checking file: ../data/can-ml/2017-subaru-forester/merged/standstill.csv
Shape: (2361472, 4)
Head:


,timestamp,arbitration_id,data_field,attack
0,1.672531e+09,881,0000000100000000000000010000000000000110101000...,0
1,1.672531e+09,328,0000101100010000101010011000011001000001000000...,0
2,1.672531e+09,329,0001011000001100001111110010001010101001000110...,0
3,1.672531e+09,320,0001101000001001101010010000011000000000000110...,0
4,1.672531e+09,321,1000000100101001101110100010100010101001000001...,0


No NaN values found.

Data types:
timestamp         float64
arbitration_id      int64
data_field         object
attack              int64
dtype: object

Descriptive statistics for numeric columns:
          timestamp  arbitration_id        attack
count  2.361472e+06    2.361472e+06  2.361472e+06
mean   1.672531e+09    4.128220e+02  4.576806e-03
std    1.523671e+02    3.345799e+02  6.749712e-02
min    1.672531e+09    2.000000e+00  0.000000e+00
25%    1.672531e+09    2.110000e+02  0.000000e+00
50%    1.672531e+09    3.280000e+02  0.000000e+00
75%    1.672532e+09    3.560000e+02  0.000000e+00
max    1.672532e+09    1.994000e+03  1.000000e+00
